# Redes Neuronais + Backpropagation

In [25]:
from itertools import product
from collections import Counter

import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

### Read Data

In [2]:
data = pd.read_csv('./data_tp1', header=None)

In [3]:
features = data.loc[:, 1:]
target = data.loc[:, 0]

In [4]:
features.shape, target.shape

((5000, 784), (5000,))

In [17]:
Counter(target)

Counter({1: 571,
         2: 530,
         9: 520,
         7: 512,
         4: 500,
         3: 500,
         8: 489,
         6: 462,
         0: 460,
         5: 456})

### Model Hiperparameters Options [[docs](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)]

In [32]:
activation = 'tanh' # Sigmoid

neuron_input = 784
neuron_output = 10

neurons_hidden = [25, 50, 100]
solver = 'sgd'
batch_sizes = [
    10, 50,         # Mini Batch
    1,              # Stochastic Gradient Descent
    target.shape[0] # Gradient Descent
]

learning_rate = 'constant'
learning_rate_inits = [0.5, 1, 10]

In [35]:
params = list()
for batch_size, learning_rate_init, neuron_hidden in product(batch_sizes, learning_rate_inits, neurons_hidden):
    params.append({
        'activation': activation,
        'hidden_layer_sizes': neuron_hidden,
        'solver': solver,
        'batch_size': batch_size,
        'learning_rate': learning_rate,
        'learning_rate_init': learning_rate_init,
        'early_stopping': False,
        'nesterovs_momentum': True,
        'momentum': .9,
        'verbose': False,
        'alpha': .0001,
        'max_iter': 256,  # number of epochs
        'shuffle': True,
        'random_state': None,
        'n_iter_no_change': 10,
        'tol': 1e-4,
    })

In [36]:
len(params)

36

### Model Instance

In [37]:
reports = list()
for curr_params in tqdm(params):
    report = dict()
    report['params'] = curr_params

    model = MLPClassifier(**curr_params)
    model.fit(features, target)

    metrics = classification_report(
        y_pred=model.predict(features),
        y_true=target,
        output_dict=True,
        zero_division=0,
    )

    report['metrics'] = metrics
    report['curves'] = {
        'loss_curve': model.loss_curve_,
        'best_loss': model.best_loss_,
        'n_iter': model.n_iter_
    }

    reports.append(report)

100%|██████████| 36/36 [46:59<00:00, 78.32s/it]   


### Evaluate Results

In [38]:
results = pd.json_normalize(reports)

In [39]:
results.to_csv('results.csv', index=False)

In [46]:
(
    results[['params.hidden_layer_sizes', 'curves.best_loss']]
    .pivot(
        values='curves.best_loss',
        columns='params.hidden_layer_sizes',
    )
    .describe()
)

params.hidden_layer_sizes,25,50,100
count,12.000000,12.000000,12.000000
mean,49.280587,291.694061,2720.351013
std,71.843700,626.904801,8759.571892
min,0.917664,0.836946,3.040507
25%,8.309022,16.197991,19.379697
50%,27.531738,32.624261,32.380846
75%,35.340187,71.007811,60.890153
max,229.388694,1991.008232,30485.104693


---